In [2]:
!python -m spacy download pt_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 2.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
# import spacy
# from spacy.matcher import PhraseMatcher

# # Carrega o modelo
# nlp = spacy.load("pt_core_news_lg")  # ou pt_core_news_sm

# # Termos capacitistas
# termos_capacitistas = [
#     "aleijado", "inválido", "defeituoso", "incapacitado", "deficiente", "ceguinho",
#     "criança excepcional", "defeituoso físico", "paraplegia", "paralisia", "pólio",
#     "doente mental", "deficiente mental", "retardado", "retardada", "retardado mental",
#     "retardo mental", "retardamento mental", "mongolóide", "mongol", "mudinho", "surdinho",
#     "surdinho-mudo", "epilético", "paralisado cerebral", "surdo-mudo", "cego-mudo",
#     "tetraplégico", "paralítico", "leproso", "lepra", "acéfalo", "acefalo", "burro", "idiota", "trouxa",
#     "estúpido", "estúpida", "demente", "morfético", "morfética", "mongoloide"
# ]

# # Cria o PhraseMatcher
# matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
# padroes = [nlp.make_doc(t) for t in termos_capacitistas]
# matcher.add("CAPACITISMO", padroes)

# # Texto de exemplo
# texto = """
# Ele é retardado. Maria está sendo tratada como mongolóide.
# João foi chamado de burro. Pedro parecia incapacitado.
# Não devemos ser estúpidos com ninguém. Você não deve tratar os outros como idiota.
# João é muito burro. Ele ficou burro demais. Esse comportamento é estúpido demais. acefalo
# """

# doc = nlp(texto)

# # Encontrar as frases que contêm os termos
# for sent in doc.sents:
#     sent_doc = nlp(sent.text)
#     matches = matcher(sent_doc)
#     if matches:
#         print(f"Expressão capacitista encontrada na frase: '{sent.text.strip()}'")

import spacy
from spacy.matcher import DependencyMatcher
from collections import defaultdict
import csv

# Caminho do arquivo (ajuste conforme necessário)
caminho_arquivo = '/content/drive/MyDrive/Mestrado/Comentarios YT/df_comentarios.csv'
caminho_saida_csv = '/content/drive/MyDrive/Mestrado/Comentarios YT/padroes_capacitismo_detectados.csv'

# Carrega modelo spaCy
nlp = spacy.load("pt_core_news_lg")

# Termos capacitistas
termos_capacitistas = [
    "aleijado", "inválido", "defeituoso", "incapacitado", "deficiente", "ceguinho",
    "criança excepcional", "defeituoso físico", "paraplegia", "paralisia", "pólio",
    "doente mental", "deficiente mental", "retardado", "retardada", "retardado mental",
    "retardo mental", "retardamento mental", "mongolóide", "mongol", "mudinho", "surdinho",
    "surdinho-mudo", "epilético", "paralisado cerebral", "surdo-mudo", "cego-mudo",
    "tetraplégico", "paralítico", "leproso", "lepra", "acéfalo", "acefalo", "burro", "idiota", "trouxa",
    "estúpido", "estúpida", "demente", "morfético", "morfética", "mongoloide", "lesado"
]

# Verbos
verbos_ligacao = ["ser", "estar", "ficar", "parecer", "tornar"]
verbos_chamar_tratar = ["chamar", "tratar"]

# Criação do matcher
matcher = DependencyMatcher(nlp.vocab)

# Padrão: sujeito + verbo + termo capacitista
pattern_caracterizacao = [
    {"RIGHT_ID": "verbo", "RIGHT_ATTRS": {"LEMMA": {"IN": verbos_ligacao}}},
    {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {
        "DEP": "nsubj", "POS": {"IN": ["PROPN", "NOUN", "PRON"]}}},
    {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "termo", "RIGHT_ATTRS": {"LOWER": {"IN": termos_capacitistas}}}
]
matcher.add("CARACTERIZACAO", [pattern_caracterizacao])

# Padrão: verbo + termo capacitista (sem sujeito)
pattern_verbo_termo = [
    {"RIGHT_ID": "verbo", "RIGHT_ATTRS": {"LEMMA": {"IN": verbos_ligacao}}},
    {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "termo", "RIGHT_ATTRS": {"LOWER": {"IN": termos_capacitistas}}}
]
matcher.add("VERBO_TERMO", [pattern_verbo_termo])

# Padrão: chamar/tratar + termo capacitista
pattern_chamar_tratar = [
    {"RIGHT_ID": "verbo", "RIGHT_ATTRS": {"LEMMA": {"IN": verbos_chamar_tratar}}},
    {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "termo", "RIGHT_ATTRS": {"LOWER": {"IN": termos_capacitistas}}}
]
matcher.add("CHAMAR_TRATAR", [pattern_chamar_tratar])

# Padrão: verbo + advérbio (intensificador) + termo capacitista
pattern_intensificado = [
    {"RIGHT_ID": "verbo", "RIGHT_ATTRS": {"LEMMA": {"IN": verbos_ligacao}}},
    {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "intens", "RIGHT_ATTRS": {"POS": "ADV"}},
    {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "termo", "RIGHT_ATTRS": {"LOWER": {"IN": termos_capacitistas}}}
]
matcher.add("INTENSIFICADO", [pattern_intensificado])

# Padrão: xingamento direto (vocativo ou ofensa isolada) → ex: "idiota!", "seu burro!"
pattern_xingamento_direto = [
    {"RIGHT_ID": "termo", "RIGHT_ATTRS": {"LOWER": {"IN": termos_capacitistas}}},
    {"LEFT_ID": "termo", "REL_OP": ">", "RIGHT_ID": "vocativo", "RIGHT_ATTRS": {"DEP": {"IN": ["intj", "discourse", "npadvmod", "nsubj"]}}}
]
matcher.add("DIRETO", [pattern_xingamento_direto])

# Padrão: comparação pejorativa → ex: "parece um retardado", "age como um mongol"
pattern_comparacao_pejorativa = [
    {"RIGHT_ID": "verbo", "RIGHT_ATTRS": {"LEMMA": {"IN": ["parecer", "agir", "falar", "pensar", "andar", "tratar"]}}},
    {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "comparador", "RIGHT_ATTRS": {"LOWER": "como"}},
    {"LEFT_ID": "comparador", "REL_OP": ">", "RIGHT_ID": "termo", "RIGHT_ATTRS": {"LOWER": {"IN": termos_capacitistas}}}
]
matcher.add("COMPARACAO_PEJORATIVA", [pattern_comparacao_pejorativa])

# Padrão: predicação com adjetivo capacitista → ex: "ele é estúpido", "ela está idiota"
pattern_adjetivo_predicativo = [
    {"RIGHT_ID": "verbo", "RIGHT_ATTRS": {"LEMMA": {"IN": verbos_ligacao}}},
    {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "adjetivo", "RIGHT_ATTRS": {
        "LOWER": {"IN": termos_capacitistas}, "POS": "ADJ"}}
]
matcher.add("ADJETIVO_PREDICATIVO", [pattern_adjetivo_predicativo])

# Contadores
contagem_padroes = defaultdict(int)
contagem_termos = defaultdict(int)
total_encontrados = 0

# Inicialização do arquivo CSV para registrar os padrões encontrados
with open(caminho_saida_csv, 'w', newline='', encoding='utf-8') as f_saida:
    writer = csv.writer(f_saida)
    writer.writerow(['Padrão', 'Frase'])

    # Lê o arquivo de comentários
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        linhas = f.readlines()

        # Processamento das frases
        for linha in linhas:
            doc = nlp(linha)
            for sent in doc.sents:
                matches = matcher(sent)
                for match_id, token_ids in matches:
                    tipo = nlp.vocab.strings[match_id]
                    palavras = [sent[i].text.lower() for i in token_ids]
                    frase_detectada = sent.text.strip()

                    # Registro no arquivo CSV
                    writer.writerow([tipo, frase_detectada])

                    print(f"\nFrase com capacitismo detectado: '{frase_detectada}'")
                    print(f" → Padrão: {tipo} | Palavras: {palavras}")
                    contagem_padroes[tipo] += 1
                    total_encontrados += 1
                    for palavra in palavras:
                        if palavra in termos_capacitistas:
                            contagem_termos[palavra] += 1
                    break  # Evita múltiplos padrões por sentença

    # Resultados
    print("\n=== CONTAGEM DE PADRÕES DETECTADOS ===")
    for tipo, qtd in contagem_padroes.items():
        print(f"{tipo}: {qtd}")
    print(f"Total geral de frases com capacitismo detectado: {total_encontrados}")

    print("\n=== CONTAGEM DE TERMOS CAPACITISTAS DETECTADOS ===")
    for termo in sorted(contagem_termos):
        print(f"{termo}: {contagem_termos[termo]}")



Frase com capacitismo detectado: 'PRESIDENCIAL 2022,Band Jornalismo,"A petrobrás é um capítulo da História Brasil e pouco se ver os interesses obscuros de particulares tentando ocultar suas motivações todos se parecer atentos no fio da miada ao ponto que quem se dá bem ñ é aquele que puxa mas aquele q pega o rolo, ninguém é bobo nem mesmo Lula que é o mais esperto de todos por ser discriminado nordestino pobre e ñ faz as coisas por fazer ou porque estudou mas por necessidade de demonstrar que ele ñ é burro como todos acham",Política,https://www.youtube.com/watch?v=WwdgWl_nmKI'
 → Padrão: DIRETO | Palavras: ['burro', 'ñ']

Frase com capacitismo detectado: 'O Lula como sempre um idiota e corrupto,Política,https://www.youtube.com/watch?v=WwdgWl_nmKI'
 → Padrão: DIRETO | Palavras: ['idiota', 'lula']

Frase com capacitismo detectado: 'O sul é e burro de carga do Brasil, tomam nossos impostos e não devolvem quase nada.'
 → Padrão: DIRETO | Palavras: ['burro', 'sul']

Frase com capacitismo d

In [ ]:
import spacy
import pandas as pd
from spacy.matcher import PhraseMatcher
from google.colab import files  # Certo para Colab

# Carrega o modelo
nlp = spacy.load("pt_core_news_lg")  # ou pt_core_news_sm

# Termos capacitistas
termos_capacitistas = [
    "aleijado", "inválido", "defeituoso", "incapacitado", "deficiente", "ceguinho",
    "criança excepcional", "defeituoso físico", "paraplegia", "paralisia", "pólio",
    "doente mental", "deficiente mental", "retardado", "retardada", "retardado mental",
    "retardo mental", "retardamento mental", "mongolóide", "mongol", "mudinho", "surdinho",
    "surdinho-mudo", "epilético", "paralisado cerebral", "surdo-mudo", "cego-mudo",
    "tetraplégico", "paralítico", "leproso", "lepra", "acéfalo", "acefalo", "burro", "idiota", "trouxa",
    "estúpido", "estúpida", "demente", "morfético", "morfética", "mongoloide"
]

# Cria o PhraseMatcher
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
padroes = [nlp.make_doc(t) for t in termos_capacitistas]
matcher.add("CAPACITISMO", padroes)

# Função para processar o CSV (Colab)
def processar_csv():
    # Fazer upload do arquivo CSV
    uploaded = files.upload()
    csv_filename = list(uploaded.keys())[0]

    # Lê apenas as primeiras 10.000 linhas
    df = pd.read_csv(csv_filename, nrows=10000)

    if 'Título' not in df.columns or 'Comentário' not in df.columns:
        raise ValueError("O CSV deve conter as colunas 'Título' e 'Comentário'.")

    resultados = []

    for index, row in df.iterrows():
        titulo = row['Título']
        comentario = row['Comentário']
        doc = nlp(comentario)
        matches = matcher(doc)

        for match_id, start, end in matches:
            span = doc[start:end]
            print(f"Título do vídeo: {titulo}")
            print(f"Comentário: {comentario}")
            print(f"Expressão capacitista encontrada: '{span.text}'\n")
            resultados.append({
                "Título": titulo,
                "Comentário": comentario,
                "Expressão encontrada": span.text
            })

    # Salvar resultados (opcional)
    if resultados:
        df_resultados = pd.DataFrame(resultados)
        df_resultados.to_csv("comentarios_capacitistas_detectados.csv", index=False)
        print("✅ Resultados salvos em 'comentarios_capacitistas_detectados.csv'.")

# Executa
processar_csv()


Saving comentarios_videos.csv to comentarios_videos.csv
Título do vídeo: DEBATE NA BAND: PRESIDENCIAL 2022
Comentário: A petrobrás é um capítulo da História Brasil e pouco se ver os interesses obscuros de particulares tentando ocultar suas motivações todos se parecer atentos no fio da miada ao ponto que quem se dá bem ñ é aquele que puxa mas aquele q pega o rolo, ninguém é bobo nem mesmo Lula que é o mais esperto de todos por ser discriminado nordestino pobre e ñ faz as coisas por fazer ou porque estudou mas por necessidade de demonstrar que ele ñ é burro como todos acham
Expressão capacitista encontrada: 'burro'

Título do vídeo: DEBATE NA BAND: PRESIDENCIAL 2022
Comentário: Toma no Cu.... Eu votei no Lula vida toda, ele só enrolou o povo brasileiro, a corrupção monstruosa que rolou no governo dele deixou o miserável continuando na miséria, desvios na parte da saúde matou foi muita gente pois sempre foi precária a situação, pobreza gera criminalidade também pois nóis se desespera e ac

In [ ]:
import spacy
from spacy.matcher import DependencyMatcher

# Carrega o modelo de linguagem
nlp = spacy.load("pt_core_news_lg")

# Lista de termos capacitistas
termos_capacitistas = [
    "aleijado", "inválido", "defeituoso", "incapacitado", "deficiente", "ceguinho",
    "criança excepcional", "defeituoso físico", "paraplegia", "paralisia", "pólio",
    "doente mental", "deficiente mental", "retardado", "retardada", "retardado mental",
    "retardo mental", "retardamento mental", "mongolóide", "mongol", "mudinho", "surdinho",
    "surdinho-mudo", "epilético", "paralisado cerebral", "surdo-mudo", "cego-mudo",
    "tetraplégico", "paralítico", "leproso", "lepra", "acéfalo", "acefalo", "burro", "idiota", "trouxa",
    "estúpido", "estúpida", "demente", "morfético", "morfética", "mongoloide"
]

# Inicializa o DependencyMatcher
matcher = DependencyMatcher(nlp.vocab)

# Geração dinâmica de padrões
for termo in termos_capacitistas:
    # 1. Verbo de ligação + termo capacitista (predicativo)
    matcher.add(f"VERBO_LIGACAO_{termo.upper()}", [
        [
            {"RIGHT_ID": "verbo", "RIGHT_ATTRS": {"LEMMA": {"IN": ["ser", "estar", "ficar", "parecer"]}}},
            {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "atributo", "RIGHT_ATTRS": {"LOWER": termo}},
        ]
    ])

    # 2. Estar + sendo + termo capacitista
    matcher.add(f"ESTAR_SENDO_{termo.upper()}", [
        [
            {"RIGHT_ID": "estar", "RIGHT_ATTRS": {"LEMMA": "estar"}},
            {"LEFT_ID": "estar", "REL_OP": ">", "RIGHT_ID": "sendo", "RIGHT_ATTRS": {"LOWER": "sendo"}},
            {"LEFT_ID": "sendo", "REL_OP": ">", "RIGHT_ID": "atributo", "RIGHT_ATTRS": {"LOWER": termo}},
        ]
    ])

    # 3. Verbo de ação + como + termo capacitista
    matcher.add(f"VERBO_COMO_{termo.upper()}", [
        [
            {"RIGHT_ID": "verbo", "RIGHT_ATTRS": {"POS": "VERB"}},
            {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "como", "RIGHT_ATTRS": {"LOWER": "como"}},
            {"LEFT_ID": "como", "REL_OP": ">", "RIGHT_ID": "atributo", "RIGHT_ATTRS": {"LOWER": termo}},
        ]
    ])

    # 4. Termo capacitista com intensificador (ex: "burro demais")
    matcher.add(f"MODIFICADOR_{termo.upper()}", [
        [
            {"RIGHT_ID": "termo", "RIGHT_ATTRS": {"LOWER": termo, "POS": "ADJ"}},
            {"LEFT_ID": "termo", "REL_OP": ">", "RIGHT_ID": "modificador", "RIGHT_ATTRS": {"POS": "ADV"}},
        ]
    ])

# Texto de exemplo
texto = """
Ele é retardado. Maria está sendo tratada como mongolóide.
João foi chamado de burro. Pedro parecia incapacitado.
Não devemos ser estúpidos com ninguém. Você não deve tratar os outros como idiota.
João é muito burro. Ele ficou burro demais. Esse comportamento é estúpido demais. acefalo
"""

doc = nlp(texto)

# Verifica sentenças e aplica matcher
for sent in doc.sents:
    matches = matcher(nlp(sent.text))
    if matches:
        for match_id, token_ids in matches:
            span = nlp(sent.text)[token_ids[0]:token_ids[-1] + 1]
            print(f"[{nlp.vocab.strings[match_id]}] Expressão capacitista encontrada: '{span.text}'")


[VERBO_LIGACAO_INCAPACITADO] Expressão capacitista encontrada: 'parecia incapacitado'
[MODIFICADOR_BURRO] Expressão capacitista encontrada: ''
[VERBO_LIGACAO_BURRO] Expressão capacitista encontrada: 'ficou burro'
